In [1]:
# Import neccessary modules
import numpy as np
import os
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [2]:
#make_Dictionary reads the email files from a folder, constructs a dictionary for all words. 
#Next, we delete words of length 1 and that are not purely alphabetical.

def make_Dictionary(root_dir):
   all_words = []
   emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)]
   for mail in emails:
        with open(mail) as m:
            for line in m:
                words = line.split()
                all_words += words
   dictionary = Counter(all_words)

   # if you have python version 3.x use commented version.
   # list_to_remove = list(dictionary)
   list_to_remove = list(dictionary)

   for item in list_to_remove:
       # remove if numerical. 
       if item.isalpha() == False:
           del dictionary[item]
       elif len(item) == 1:
               del dictionary[item]
    # consider only most 3000 common words in dictionary.

   dictionary = dictionary.most_common(3000)

   return dictionary

In [3]:
#Now we have dictionary function. Next with the help of dictionary, we generate a label and word frequency matrix
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 0;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i,line in enumerate(fi):
        if i == 2:
          words = line.split()
          for word in words:
            wordID = 0
            for i,d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens) - 1]
      if lastToken.startswith("spmsg"):
          train_labels[docID] = 1;
          count = count + 1
      docID = docID + 1
  return features_matrix, train_labels

In [6]:
# Loading datasets
TRAIN_DIR = "train-mails"
TEST_DIR = "test-mails"

dictionary = make_Dictionary(TRAIN_DIR)

# using functions mentioned above.
print ("reading and processing emails from file.")
features_matrix, labels = extract_features(TRAIN_DIR)
test_feature_matrix, test_labels = extract_features(TEST_DIR)



reading and processing emails from file.


In [7]:
#Gaussian is used in classification and it assumes that features follow a normal distribution.
model = GaussianNB()
print ("Training model.")
#train model
model.fit(features_matrix, labels)

#predict
predicted_labels = model.predict(test_feature_matrix)

print ("FINISHED classifying. accuracy score : ")
print (accuracy_score(test_labels, predicted_labels))

Training model.
FINISHED classifying. accuracy score : 
0.9653846153846154
